# Utilisation des transformers pré-entrainés et fine-tuning
Dans les tps précédents, on a vu comment créer et entraîner un transformer from scratch. Mais souvent, on n'entraine pas un transformer from scratch, mais on utilise un transformer pré-entrainé et on fait du fine-tuning. Le fine-tuning, aussi appelé transfert learning, consiste à prendre un modèle pré-entrainé (dans notre cas GPT-2) et à l'entraîner davantage sur un dataset spécifique. L'idée est que le modèle a déjà appris des caractéristiques générales lors de son entraînement initial, et maintenant, avec le fine-tuning, il peut apprendre des caractéristiques spécifiques à notre problème.

### N'oubliez pas de mettre le runtime en GPU si vous êtes sur colab

## Utilisation de GPT-2
Ici, on va utiliser GPT-2. GPT-2 est pré-entrainé sur un dataset de 8 millions de pages web.

Pour ça, on va utiliser la librairie transformers de HuggingFace. Cette librairie permet d'utiliser des modèles pré-entrainés et de les fine-tuner. Elle permet aussi de télécharger des modèles pré-entrainés.
On va donc télécharger le modèle GPT-2 et l'utiliser pour générer du texte.

Pour installer la librairie, il faut faire `pip install transformers`

Pour télécharger le modèle, il faut faire `from transformers import GPT2LMHeadModel, AutoTokenizer`

Il existe plusieurs classe de GPT2, par exemple GPT2Model, GPT2LMHeadModel, GPT2ForSequenceClassification, etc.

GPT2Model ne contient pas la dernière couche linéaire qui permet de faire la prédiction du mot suivant. GPT2LMHeadModel contient cette dernière couche linéaire. GPT2ForSequenceClassification prend le dernier token et fait une classification dessus.

On va utiliser GPT2LMHeadModel, car on veut générer du texte.

Pour avoir plus d'informations sur les modèles, il faut aller sur le site de HuggingFace : https://huggingface.co/transformers/model_doc/gpt2.html

AutoTokenizer permet de télécharger le tokenizer associé au modèle.

### Telechargement du modèle et du tokenizer

In [ ]:
from transformers import GPT2LMHeadModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

Remarque: dans certains tutos, ils utilisent `AutoModelForCausalLM` au lieu de `GPT2LMHeadModel`. C'est la même chose, c'est juste que `AutoModelForCausalLM` est plus général, car il peut être utilisé pour d'autres modèles que GPT-2.

### Exemple d'utilisation du tokenizer

In [ ]:
text = "Hello, my dog is cute"
tokens = tokenizer(text)
print(tokens)

tokens = tokenizer(text, return_tensors="pt")
print(tokens)

### Exemple d'utilisation du modèle

In [ ]:
input_ids = tokens["input_ids"]
attention_mask = tokens["attention_mask"]

output = model(input_ids, attention_mask=attention_mask)
print(output)

Comme vous pouvez voir, `output` est un objet de classe `CausalLMOutputWithCrossAttentions`, qui contient plusieurs informations.
Si vous lancez la cellule suivante, vous aurez plus d'informations sur cette classe.

In [ ]:
help(output)

On peut voir que `output` contient `loss`, `logits`, `past_key_values`, `hidden_states`, `attentions`, `cross_attentions`.
`loss` est `None` si on ne lui donne pas `labels` en entrée.
`logits` est un tenseur de taille `[batch_size, sequence_length, vocab_size]`, qui correspond aux scores du mot suivant pour chaque mot du vocabulaire. Si on applique un softmax à `logits`, on obtient la probabilité du mot suivant pour chaque mot du vocabulaire.

Si on prend l'argmax du dernier token de `logits`, on obtient le mot suivant le plus probable.

Vous pouvez regarder par vous-même les autres attributs de `output`.

Exemple :

In [ ]:
print(output.loss)

Vous remarquez que `loss` est `None`, car on n'a pas donné `labels` en entrée. Pour donner `labels` en entrée, il faut faire ajouter dans le dictionnaire `tokens` la clé "labels" avec comme valeur `input_ids`.
Par exemple:
```python
tokens["labels"] = input_ids
```

En effet, on n'a pas besoin de "shift" les labels, car le shifting est déjà fait dans le modèle GPT-2.
Pour rappel: le shifting consiste à décaler les labels d'un token vers la gauche. Par exemple, si on a le texte "Hello, my dog is cute", le shifting consiste à avoir comme labels "my dog is cute" et comme input "Hello, my dog is".

In [ ]:
print(type(output.logits))
print(output.logits.shape)

On peut voir que `logits` est un tenseur de taille `[1, 6, 50257]`. 1 correspond à la taille du batch, 6 correspond à la taille de la séquence, et 50257 correspond à la taille du vocabulaire.

### Exercice:
Sachant que `tokenizer.bos_token` correspond au token de début de séquence (**B**eginning **O**f **S**entence), `tokenizer.eos_token` correspond au token de fin de séquence (**E**nd **O**f **S**entence), et que `tokenizer.decode` permet de transformer des ids en texte, esayez de générer du texte avec le modèle GPT-2.

In [ ]:
print(tokenizer.bos_token)
print(tokenizer.eos_token)
print(tokenizer.decode([15496, 11, 616, 3290, 318, 13779]))

Remarquez que pour GPT2, bos_token = eos_token

In [ ]:
# Votre code ici

## Fine-tuning de GPT-2
On pourrait écrire nous-même le code pour faire du fine-tuning (c'est juste un entraînement classique, c'est juste que les poids de départ sont ceux du modèle pré-entrainé), , mais heureusement, la librairie transformers le fait pour nous. Il faut utiliser `Trainer` et `TrainingArguments` de la librairie transformers.

### Exercice: Utilisation de `distilgpt2` pour faire du fine-tuning
En fait, GPT2 est beaucoup trop gros pour être utilisé sur colab. On va donc utiliser `distilgpt2`, qui est une version plus petite de GPT2. `distilgpt2` a été pré-entraîné sur le même dataset que GPT2, mais il est 2 fois plus petit que GPT2.

Pour utiliser "distilgpt2", il suffit de remplacer "gpt2" par "distilgpt2" quand vous faites `from_pretrained("gpt2")`.

In [ ]:
# Utilisation de distilgpt2
# votre code ici


### Preparation du dataset
On va télécharger le même dataset qu'on a utilisé pour entraîner notre modèle from scratch.

In [ ]:
import requests

url = "https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/colab_train.txt"
r = requests.get(url)
dataset = r.text

# save file
with open("colab_train.txt", "w") as f:
	f.write(dataset)

url = "https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/transfo/colab_train_prompts.txt"
r = requests.get(url)
dataset_prompt = r.text

# save file
with open("colab_train_prompts.txt", "w") as f:
	f.write(dataset_prompt)

Dans le fichier `colab_train_prompts.txt`, chaque ligne correspond à un prompt, et dans le fichier `colab_train.txt`, chaque ligne correspond à l'histoire associée au prompt de `colab_train_prompts.txt`.

### Exercice: Création du dataset
Maintenant, il faut créer le dataset pour le fine-tuning. On va creer un custom dataset avec la classe `Dataset` de pytorch. On va créer un dataset qui prend en entrée un prompt et qui renvoie l'histoire associée au prompt.

Pour créer un dataset avec la classe `Dataset` de pytorch, il faut créer une classe qui hérite de `Dataset` et qui a comme attributs `__len__` et `__getitem__`.
`__len__` renvoie la taille du dataset, et `__getitem__` renvoie l'élément à l'index donné en entrée.

Dans notre cas, `__getitem__` doit renvoyer un dictionnaire avec les clés "input_ids". Comme vous avez vu dans la partie précédente, "input_ids" et il correspond aux ids de "{prompt} {histoire}".
Pas besoin de créer les "labels".

Il faut donc combiner les ids du prompt et de l'histoire pour avoir les ids de "{prompt} {histoire}".

Si la longueur de "{prompt} {histoire}" est supérieure à 1024, il faut couper "{prompt} {histoire}" en plusieurs morceaux de taille 1024, car GPT-2 ne peut pas prendre en entrée des séquences de taille supérieure à 1024.

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
	def __init__(self):
		# votre code ici
		# lire le fichier colab_train.txt et colab_train_prompts.txt et convertir en liste de str
		# vous devriez avoir 2 listes de même taille
		pass

	def __len__(self):
		# votre code ici
		# retourner len(liste de str)
		pass

	def __getitem__(self, idx):
		# votre code ici
		# tokenizer le prompt et l'histoire à l'index idx
		# concatener les input_ids du prompt et de l'histoire
		# renvoyer un dictionnaire avec les clés "input_ids"
		pass

### Utilisation de `Trainer` et `TrainingArguments`

Maintenant, on va utiliser `Trainer` et `TrainingArguments` pour faire du fine-tuning.
`TrainingArguments` permet de définir les arguments d'entraînement, comme le nombre d'epochs, la taille du batch, etc.
`Trainer` permet de faire l'entraînement, et il prend en entrée un `TrainingArguments`, un `model`, un `data_collator`, et un `train_dataset`.

Un `data_collator` est un objet `DataCollator` de la librairie transformers. Il permet de combiner les données en batch (ce qui est pratique car ça nous évite de pad nous-même les données). Il existe plusieurs `DataCollator`, et on va utiliser `DataCollatorForLanguageModeling`, qui permet de combiner les données en batch pour faire du language modeling.

Pour créer un `DataCollatorForLanguageModeling`, on lance la cellule suivante:

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Pour tester si le `data_collator` et le `dataset` fonctionnent bien ensemble, on peut faire:

In [ ]:
dataset = CustomDataset()
out = data_collator([dataset[i]] for i in range(5))

for key in out:
	print(f"{key} shape: {out[key].shape}")

Vous devriez avoir comme sortie:
```
input_ids shape: torch.Size([5, 1024])
attention_mask shape: torch.Size([5, 1024])
labels shape: torch.Size([5, 1024])
```

On obtient bien des batchs de taille 5, avec des input_ids, des attention_mask et des labels de taille 1024.

### Création du `TrainingArguments` et du `Trainer`

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
	output_dir="./gpt2",
	num_train_epochs=1,
	per_device_train_batch_size=4,
	save_steps=10,
	save_total_limit=2,
)

trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=dataset,
)

### Entraînement du modèle

In [ ]:
trainer.train()

# Exercice: Génération de texte
Maintenant que le modèle est entraîné, on va générer du texte avec le modèle fine-tuné.
Vous pouvez utiliser `model` comme on l'a fait précédemment.

Essayez de générer l'histoire du premier prompt du fichier `colab_train_prompts.txt`.

In [ ]:
# votre code ici

# Utilisation de BERT
BERT est un autre transformer pour faire du NLP. Il est plus utilisé pour faire de la classification de texte que pour faire de la génération de texte.

BERT veut dire Bidirectional Encoder Representations from Transformers. Il est bidirectionnel car lors du pretraining, on ne met pas de causal mask, donc le modèle peut voir les mots futurs, contrairement à un modèle comme GPT-2 qui ne peut que voir les mots précédents (et donc unidirectionel).

Lors du pretraining, on utilise 2 tâches: Masked Language Modeling (MLM) et Next Sentence Prediction (NSP).
Le MLM consiste à masquer des mots dans une phrase et à faire prédire les mots masqués (en utilisant les mots précédents et les mots suivants du mot masqué). L'intuition est que le modèle doit comprendre le contexte pour pouvoir prédire le mot masqué.

Le NSP consiste à donner 2 phrases en entrée et à faire prédire si la deuxième phrase suit la première phrase.
Idem, l'intuition est que le modèle doit comprendre le contexte pour pouvoir prédire si la deuxième phrase suit la première phrase.

Pour plus d'informations sur BERT, vous pouvez lire le papier original: https://arxiv.org/pdf/1810.04805.pdf

Dans ce TP, on va utilisé une versio fine-tunée de BERT pour faire du Question Answering (QA). Le QA consiste à donner une question et une réponse en entrée et à faire prédire le "score" de cohérence entre la question et la réponse.

Donc par exemple si on a la question "What color is the sky?" et deux réponses "The sky is blue" et "I love orange", le modèle doit prédire un score plus élevé pour "The sky is blue" que pour "I love orange".

Pour ça, on va utiliser `multi-qa-mpnet-base-dot-v1`, qui est un modèle fine-tuné de BERT pour faire du QA.
On va passer en entrée la question dans le modèle, pour avoir un vecteur de taille 768, et représente la question.
On va faire la même chose pour les réponses.

On a alors un vecteur $Q$ qui représente la question et deux vecteurs $R_1$ et $R_2$ qui représentent les réponses.
Ensuite, on va calculer le produit scalaire entre $Q$ et $R_1$ et entre $Q$ et $R_2$ pour avoir 2 scores $S_1$ et $S_2$.

Puis on compare $S_1$ et $S_2$ pour savoir quelle réponse est la plus cohérente avec la question.


Vous pouvez avoir plus d'informations sur les senteces transformers ici: https://www.sbert.net/docs/pretrained_models.html


### Utilisation de `multi-qa-mpnet-base-dot-v1`
Pour utiliser `multi-qa-mpnet-base-dot-v1`, il faut installer la librairie `sentence-transformers` avec `pip install sentence-transformers`.


In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

Pour encoder une phrase, il suffit de faire:

In [ ]:
print(model.encode("How are you?"))

### Exercice: Calcul du vecteur embedding de la question et des réponses et calcul du score

In [ ]:

question_embedding = ...
answer_embedding_1 = ...
answer_embedding_2 = ...

score_1 = ...
score_2 = ...

print(f"score 1: {score_1}")
print(f"score 2: {score_2}")